Generate an isoscape from a variational inferenence model

In [ ]:
MODEL_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/random_boosted_isorix_carbon_ensemble.tf" #@param
TRANSFORMER_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/random_boosted_isorix_carbon_ensemble.pkl" #@param
OUTPUT_RASTER_NAME = "amazon_test_old_res" #@param
USE_LOCAL_DRIVE = False #@param {type:"boolean"}

# Number of pixels on each dimension
RESOLUTION_X = 300 #@param
RESOLUTION_Y = 300 #@param
AMAZON_ONLY = False #@param {type: "boolean"}

In [ ]:
#@title Imports and modules.

import tensorflow as tf
import joblib
import os
import sys
from matplotlib import rc

rc('animation', html='jshtml')

!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_import_common()

In [ ]:
import model
import raster

# Access data stored on Google Drive if not reading data locally.
if not USE_LOCAL_DRIVE:
  raster.mount_gdrive()

# Import Tensorflow model and scalers

In [ ]:
vi_model = model.TFModel(MODEL_SAVE_LOCATION, TRANSFORMER_SAVE_LOCATION)

# Note: These need to be in the same as the columns used to train the model.
required_geotiffs = [
    'VPD', 'RH', 'PET', 'DEM', 'PA',
    'Mean Annual Temperature','Mean Annual Precipitation',
    'Iso_Oxi_Stack_mean_TERZER',
    'isoscape_fullmodel_d18O_prec_REGRESSION',
    'brisoscape_mean_ISORIX',
    'd13C_cel_mean',
    'd13C_cel_var',
    'ordinary_kriging_linear_d18O_predicted_mean',
    'ordinary_kriging_linear_d18O_predicted_variance']

In [ ]:
raster.generate_isoscapes_from_variational_model(
  vi_model, required_geotiffs, RESOLUTION_X, RESOLUTION_Y, OUTPUT_RASTER_NAME, amazon_only=AMAZON_ONLY)

# Rendering

## Means

In [ ]:
means_anim = raster.animate(raster.load_raster(raster.get_raster_path(OUTPUT_RASTER_NAME+".tiff"), use_only_band_index=0), 1, 1)
means_anim

# Vars

In [ ]:
vars_anim = raster.animate(raster.load_raster(raster.get_raster_path(OUTPUT_RASTER_NAME+".tiff"), use_only_band_index=1), 1, 1)
vars_anim